# GEOPHYS 257 (Winter 2023)
[//]: <> (Notebook Author: Thomas Cullison, Stanford University, Jan. 2023)

## Multicore Parallelization of Matrix Multiplication Using Numba

In this lab we will be using Numba to accelerate matrix-matrix multiplications by exploiting parallelism. Even most laptops today have Multicore CPUs, where a *core* is a microprocessor, and each core is usually a copy of the each other core. Accelerating the marrix-matrix multiplication operation is a good analog to accelerating other types of operators and computationally intense kernels, codes, and algorithms. Furthermore, the structure of matricies makes matrix-matrix multiplication a good place start learning how to parallelize code.


## External Resources
If you have any question regarding some specific Python functionality you can consult the official [Python documenation](http://docs.python.org/3/).

* [Numba](https://numba.readthedocs.io/en/stable/index.html): Documentation
* [Numba in 30 min](https://youtu.be/DPyPmoeUdcE): Conference presentation video


## Required Preperation
Please watch the following videos before starting the lab (each is pretty short):
* [Introduction to Parallel Computing](https://youtu.be/RNVIcm8-6RE)
* [Amdahl's Law](https://youtu.be/Axx2xuB-Xuo)
* [CPU Caching](https://youtu.be/KmairurdiaY)
* [Pipelining](https://youtu.be/zPmfprtdzCE)
* [Instruction Level Parallelism](https://youtu.be/ZoDUI3gTkDI)
* [Introduction to SIMD](https://youtu.be/o_n4AKwdfiA)


## Exercise 0

Please run the following cells.  Examine the result of the example function that makes use of the *timer* wrapper. Also please use this timer wrapper (defined below) for all the exercises that follow.

### Load python modules 
#### (Note: you may need to install some Python packages for the modules below, e.g. Numba)

In [1]:
import numba
import numpy as np


from functools import wraps
from time import time, sleep
from itertools import permutations

### Define a function-decorator for timing the runtime of your functions

Below is some code that you can use to wrap your functions so that you can time them individually.  The function defined immediately below the *timer()* is an example of how to use the warpper. In the cell that follows, execute this example function. (Note, for this timer, were are making use of something called *decorators*, but a discussion about this feature is outside the scope of this class.)


In [2]:
# defining a function decorator for timing other functions
def mytimer(func):
    @wraps(func)
    def wrap(*args, **kwargs):
        t_start = time() # Students look here
        result = func(*args, **kwargs)
        t_end = time() # Students also look here. This is how you can time things inside functions/classes
        print(f'Function: \'{func.__name__}({kwargs})\' executed in {(t_end-t_start):6.5f}s\n')
        return result
    return wrap
    

# Example of how to use. NOTE the "@mytimer" stated just above the function definition
@mytimer
def example_sum_timer_wrap(N):
    """ Sum the squares of the intergers, i, contained in [1-N] """
    return np.sum(np.arange(1,N+1)**2)

### Run the example function

Run the example function for each of the following instances of $N = 10^5, 10^6, 10^7, 10^8$. Please examine the results, in particular, how the runtime changes with respect to $N$.


**Answer** the following questions in the markdown cell that follow the code. 
* For each factor-of-ten increase in $N$, roughly how much longer was the runtime of the function?
* Does this slowdown in the runtime make sense? Why or why not?

In [5]:
# Call the example function above for each value of N (try making one-call first, then loop)

N_list = 10 ** np.arange(5, 9, dtype=np.int32)

for N in N_list:
    example_sum_timer_wrap(N = N)

Function: 'example_sum_timer_wrap({'N': 100000})' executed in 0.00106s

Function: 'example_sum_timer_wrap({'N': 1000000})' executed in 0.00971s

Function: 'example_sum_timer_wrap({'N': 10000000})' executed in 0.09520s

Function: 'example_sum_timer_wrap({'N': 100000000})' executed in 0.94551s



## Discussion for Exercise 0 questions
* For each factor-of-10 increase in $N$, the function runtime is also roughly 10 times longer.
* This linear $O(N)$ slowdown makes sense. Within the function it creates a numpy array with size $N$, and the number of math operations also increases linearly with $N$.

## Exercise 1: Naive matrix multiplication: 

For this exercise, we will write our own matrx-matrix multiplication function.  We are goint to be naive about it, so we want to loop over individual indices, instead of using slicing (in the next section, we will parallelize this code and we want to see how well Numba does at speeding up the naive code).

### Tasks for this exercise

1. Write a function with that calculates matrix-matrix multiplication such that $C = A\cdot B$, where $A$, $B$, and $C$ are 2D numpy arrays. Make the dtype for the $C$ matrix the same as the dtype for $A$. Below is a stencil you can start with. Test your results for accuracy and performance against the np.dot() function. To time the np.dot() function, you can wrap it in another function and use the *mytimer* wrapper; however, please copy the code for testing the A and B dimensions into your function wrapper for the np.dot() function. This keeps the runtime comparison as a more "apples-to-apples" like comparison.

```python
@mytimer
def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zeros 
    # and that has the appropriate dimensions such that C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in range(<val>):
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
<br>

2. Define your functions in the cell below.
3. Test and compare the accuracy and runtime of your functions in the next cell.
    - Test with non-square Matrices: $A \in \mathbb{R}^{N\times K}$ and $B \in \mathbb{R}^{K \times M}$. With $N = 64$, $K = 32$, and $M = 128$.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 64, 128,$ and $256$.
    - Test the following three cases:
        - Case-1. $A$ and $B$ **both** have dtype=np.**float32** (make sure that $C$ is also of dtype=np.**float32**)
        - Case-2. $A$ has dtype=np.**float64**, but $B$ has dtype=np.**float32**
        - Case-3. $A$ and $B$ **both** have dtype=np.**float64** (make sure that $C$ is also of dtype=np.**float64**)
    - For all three case above:
        - Calculate and show the error by computing the sum of the difference between the $C$ matrices computed from numpy.dot() and your my_naive_matmul() functions. Assume that numpy.dot() is correct
        - Calculate and show the *speedup* that the faster function has versus the slower function.
        - Comment on the which of the three cases is fastest, and comment on what the speedup of the fastest case is and why it is the fastest case.
4. Create your matrices using random numbers. An example is shown below (feel free to copy this).

```python
A = np.random.rand(N, K)
B = np.random.rand("for-you-to-figure-out")
```
<br>

### Write function definitions for Exercise 1 below

In [4]:
# Define your functions for Exercise 1 here.

def my_naive_matmul(A:np.ndarray, B:np.ndarray):
    """ This is a naive matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    # 2. 
    # construct a 2D numpy array for matrix C, that is filled with zeros 
    # and that has the appropriate dimensions such that C=A*B is a valid 
    # equation and operation
    C = np.zeros((N, M), dtype=A.dtype)
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    for i in range(N):
        for k in range(K):
            for j in range(M):
                C[i, j] += A[i, k] * B[k, j]
    
    # 4
    # return the 2D numpy array for C
    return C

def np_matmul(A:np.ndarray, B:np.ndarray):
    """ This is the matrix-multiplication function using numpy """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
        
    # 2. 
    # construct a 2D numpy array for matrix C, that is filled with zeros 
    # and that has the appropriate dimensions such that C=A*B is a valid 
    # equation and operation
    C = np.zeros((N, M), dtype=A.dtype)
    
    # 3.
    # calculate matrix-multiplication with np.dot()
    np.dot(A, B, out=C)
    
    # 4
    # return the 2D numpy array for C
    return C

def compare_matmul(C1:np.ndarray, C2:np.ndarray):
    """ Calculate absolute error between two matrices """
    return np.sum(np.abs(C1 - C2))

In [7]:
# Function for testing

def test_naive_matmul(shape):
    """ This function compares naive matrix-multiplication with np.dot() """

    # 1.
    # Check the shape of matrix
    if shape[0] == shape[1] and shape[1] == shape[2]:
        print('Square matrices, N = %d' %shape[0])
    else:
        print('Non-square matrices, N = %d, K = %d, M = %d' %(shape[0], shape[1], shape[2]))
        
    # 2.
    # Generate random matrices
    A0 = np.random.rand(shape[0], shape[1])
    B0 = np.random.rand(shape[1], shape[2])
    
    # 3.
    # Loop over three cases with different float data types
    for case in range(3):
        
        # 3.1.
        # Float types of matrices
        if case == 0:
            print('\n#1: Both A and B are float32')
            A = A0.astype(np.float32)
            B = B0.astype(np.float32)

        elif case == 1:
            print('\n#2: A is float64, and B is float32')
            A = A0.astype(np.float64)
            B = B0.astype(np.float32)

        elif case == 2:
            print('\n#3: Both A and B are float64')
            A = A0.astype(np.float64)
            B = B0.astype(np.float64)

        # 3.2.
        # Time functions: Naive matrix-multiplication and np.dot
        t1_start = time()
        C1 = my_naive_matmul(A, B)
        t1_end = time()
        t1 = t1_end - t1_start

        t2_start = time()
        C2 = np_matmul(A, B)
        t2_end = time()
        t2 = t2_end - t2_start

        # 3.3.
        # Calculate speed-up and accuracy
        if (t1 >= t2):
            print('Numpy function is faster')
            speedup = t1 / t2
        else:
            print('Naive matmul is faster')
            speedup = t2 / t1
        
        print('Naive runtime: %.6f s' %t1)
        print('Numpy runtime: %.6f s' %t2)
        print('Speed-up factor: %.3f' %speedup)
        print('Error: %s' %compare_matmul(C1, C2))

<br>

### Write the test codes for Exercise 1 in the cell below.

In [16]:
# Non-square matrices

N = 64
K = 32
M = 128

test_naive_matmul([N, K, M])

Non-square matrices, N = 64, K = 32, M = 128

#1: Both A and B are float32
Numpy function is faster
Naive runtime: 0.169088 s
Numpy runtime: 0.000060 s
Speed-up factor: 2814.310
Error: 0.0022149086

#2: A is float64, and B is float32
Numpy function is faster
Naive runtime: 0.201134 s
Numpy runtime: 0.000084 s
Speed-up factor: 2396.642
Error: 3.4914293678411923e-12

#3: Both A and B are float64
Numpy function is faster
Naive runtime: 0.175261 s
Numpy runtime: 0.000072 s
Speed-up factor: 2434.099
Error: 3.4585667663122877e-12


In [19]:
# Square matrices

for N in np.array([64, 128, 256]):
    test_naive_matmul([N, N, N])
    print('\n%s\n' %("-" * 75))

Square matrices, N = 64

#1: Both A and B are float32
Numpy function is faster
Naive runtime: 0.177340 s
Numpy runtime: 0.000056 s
Speed-up factor: 3151.767
Error: 0.0027246475

#2: A is float64, and B is float32
Numpy function is faster
Naive runtime: 0.201891 s
Numpy runtime: 0.000110 s
Speed-up factor: 1836.859
Error: 3.424815986363683e-12

#3: Both A and B are float64
Numpy function is faster
Naive runtime: 0.168809 s
Numpy runtime: 0.000071 s
Speed-up factor: 2383.963
Error: 3.4710012641880894e-12

---------------------------------------------------------------------------

Square matrices, N = 128

#1: Both A and B are float32
Numpy function is faster
Naive runtime: 1.333306 s
Numpy runtime: 0.000244 s
Speed-up factor: 5461.221
Error: 0.042549133

#2: A is float64, and B is float32
Numpy function is faster
Naive runtime: 1.558181 s
Numpy runtime: 0.000434 s
Speed-up factor: 3590.926
Error: 2.772182483568031e-11

#3: Both A and B are float64
Numpy function is faster
Naive runtime:

## Discussion for Exercise 1 questions
* Numpy function np.dot() is much faster than naive matrix multiplication. With larger matrix sizes, the speedup of np.dot() becomes more significant. This is probably because with larger matrix sizes, the parallel part of the program becomes more dominant than the serial part.
* Typically, when both matrices are in float32 type, the runtime is the shortest. This is because float32 type has less significant digits (precision) and the operation is applied to fewer bytes of data. 
* Similarly, this case has the largest speedup, but not so different from the case where both matrices are in float64 type. Potentially this is because with fewer digits, np.dot() optimization performs better.
* An interesting feature is that if two matrices have different float types, the runtime is usually the longest and the speedup is the smallest. This can be due to that before multiplication, there is an extra step to homogenize the inputs into the same type.

## Exercise 2: Using Numba to speed up matrix multiplication: 

For this exercise, numba to speedup our matrx-matrix multiplication function below. However, there is an interesting twist to this exercise. We will write six versions of the naive function you wrote above. One function for each of the six possible permutations of three loops used to calculate the multplication (i.e. ijk, ikj, jik, etc.)

### The tasks for this exercise:
1. Make six copies your code for the my_naive_matmul(), one copy for each of the possible permutations and define them in the cell bellow:
    - One of these function will have the same loop order as the my_naive_matmul() function. 
    - However name these functions: numba_mul_\<perm\>(), where \<perm\> should be replaced by the specific loop order of the function.
2. In the cell that follows your function definitions, test and compare the accuracy and runtime of your functions against the numpy.dot() function.
    - Test with a square Matrices: $A,B \in \mathbb{R}^{N\times N}$. For the cases when $N = 128, 256,$ and $512$.
    - For each matrix set their dtype as: dtype=np.float64
    - Calculate and show the error between your functions and the numpy.dot() function. (Same as in Exercise 1.)
    - Calculate and show the *speedup* that the fastest function has versus all the other functions.
    - You should notice that one of your permutation functions is faster than the others. For this case show the following:
        - Calculate and show the *speedup* that the fastest permutation function has versus the my_naive_matmul().
        - Calculate and show the *speedup* that the fastest permutation function has when $A$, $B$, and $C$ are all of dtype=np.float64 vs all are of dtype=np.float32.
3. Create your matrices using random numbers. (Same as in Exercise 1.) 
4. For each function, you need to add a function decorator for numba. Numba will *JIT* the function (**J**ust **I**n **T**ime compilation). 
    - Use the flag to keep a "cache" of the compiled code (not to be confused with CPU cache). **Note**: to get accurate timings, you will need to run your tests **twice**, because during the first run, the code is compiled and this compile time will be included in your runtime. After the first run, the compiled binary will be stored, so consecutive runs will be faster.
    - Use the flag to use *fast math*
    - Use the flag to disable the Python Global Interpretor Lock (GIL).
    - The code below should get you started, but it is incomplete.
    
```python   
    
@mytimer
@numba.njit(<flagname_caching>=<flag>, <flagname_fast_math>=<flag>, <flagname_no_gil>=<flag>)
def numba_mul_<perm>(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """ 
    
    # 1. 
    # check that A and B have appropriate dimensions for multiplication
    
    # 2. 
    # construnct a 2D numpy array for matrix C, that is filled with zerros 
    # and that has the appropriate dimensions such taht C=A*B is a valid 
    # equation and operation
    
    # 3
    # Write three nested for loops, with indices, i,j,k to solve the matrix-multiplication
    # e.g.
    # for i in numba.prange(<val>): # !!!! LOOK HERE !!!!
    #   for j in range(<val>):
    #     for k in range(<val>):
    #       C[<c_row_index>,<c_col_index>] += A[<a_row_index>,<a_col_index>]*B[<b_row_index>,<b_col_index>]
    
    # 4
    # return the 2D numpy array for C
    return C
```
    
5. Discuss your results in the markdown cell that follows your codes include in your discussion remarks about the questions asked in the markdown cell.


<br>

### Write function definitions for Exercise 2 below

In [20]:
# Define your functions for Exercise 2 here.

@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_matmul_1(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """  
    """ Index order: ijk (i loops over N, j loops over M, k loops over K)"""
    
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    C = np.zeros((N, M), dtype=A.dtype)
    
    for i in numba.prange(N):
        for j in range(M):
            for k in range(K):
                C[i, j] += A[i, k] * B[k, j]
    
    return C


@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_matmul_2(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """  
    """ Index order: ikj (i loops over N, j loops over M, k loops over K)"""
    
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    C = np.zeros((N, M), dtype=A.dtype)
    
    for i in numba.prange(N):
        for k in range(K):
            for j in range(M):
                C[i, j] += A[i, k] * B[k, j]
    
    return C


@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_matmul_3(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """  
    """ Index order: jik (i loops over N, j loops over M, k loops over K)"""
    
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    C = np.zeros((N, M), dtype=A.dtype)
    
    for j in numba.prange(M):
        for i in range(N):
            for k in range(K):
                C[i, j] += A[i, k] * B[k, j]
    
    return C


@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_matmul_4(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """  
    """ Index order: jki (i loops over N, j loops over M, k loops over K)"""
    
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    C = np.zeros((N, M), dtype=A.dtype)
    
    for j in numba.prange(M):
        for k in range(K):
            for i in range(N):
                C[i, j] += A[i, k] * B[k, j]
    
    return C


@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_matmul_5(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """  
    """ Index order: kij (i loops over N, j loops over M, k loops over K)"""
    
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    C = np.zeros((N, M), dtype=A.dtype)
    
    for k in numba.prange(K):
        for i in range(N):
            for j in range(M):
                C[i, j] += A[i, k] * B[k, j]
    
    return C


@numba.njit(cache=True, fastmath=True, nogil=True)
def numba_matmul_6(A:np.ndarray, B:np.ndarray):
    """ This is a Numba accelerated matrix-multiplication function """  
    """ Index order: kji (i loops over N, j loops over M, k loops over K)"""
    
    N, K = A.shape
    _K, M = B.shape
    
    if K != _K:
        raise ValueError('Matrices dimensions do not match for multiplication!')
    
    C = np.zeros((N, M), dtype=A.dtype)
    
    for k in numba.prange(K):
        for j in range(M):
            for i in range(N):
                C[i, j] += A[i, k] * B[k, j]
    
    return C

In [21]:
# Function for testing

def test_numba_matmul(shape, dtype=np.float64):
    """ This function compares numba matrix-multiplication with np.dot() """
    """ Numba functions have 6 permutations of indices """

    # 1.
    # Check the shape of matrix
    if shape[0] == shape[1] and shape[1] == shape[2]:
        print('Square matrices, N = %d' %shape[0])
    else:
        print('Non-square matrices, N = %d, K = %d, M = %d' %(shape[i] for i in range(3)))
        
    print()
    
    # 2.
    # Generate random matrices
    A = np.random.rand(shape[0], shape[1]).astype(dtype)
    B = np.random.rand(shape[1], shape[2]).astype(dtype)
    
    # 3. 
    # Reference result from np.dot()
    t_start = time()
    C_ref = np_matmul(A, B)
    t_end = time()
    t_ref = t_end - t_start

    # 4.
    # Loop over all numba functions. Record runtime with respect to np.dot()
    numba_func_list = [numba_matmul_1, numba_matmul_2, numba_matmul_3, \
                       numba_matmul_4, numba_matmul_5, numba_matmul_6]
    Nfunc = len(numba_func_list)
    numba_runtime = np.zeros((Nfunc, 1))

    for i in range(Nfunc):
        
        t_start = time()
        C = numba_func_list[i](A, B)
        t_end = time()
        
        numba_runtime[i] = t_end - t_start
    
    # 5.
    # Output error
    print('Error: %s' %(compare_matmul(C, C_ref)))
    print()
    
    # 6.
    # Find the fastest numba method and compare with np.dot()
    ind_best = np.argmin(numba_runtime)
    numba_runtime_best = numba_runtime[ind_best]
    
    if (numba_runtime_best >= t_ref):
        print('Fastest method: np.dot()')
        print('Best Numba permutation: %s' %(numba_func_list[ind_best].__name__))
        print('Speed-up over Numba functions: %s' %(", ".join("%.2f" % obj for obj in numba_runtime / t_ref)))
    else:
        print('Fastest method: %s' %(numba_func_list[ind_best].__name__))
        print('Speed-up over np.dot: %s' %(t_ref / numba_runtime_best))
        print('Speed-up over Numba functions: %s' %(", ".join("%.2f" % obj for obj in numba_runtime / numba_runtime_best)))

In [22]:
# Function for testing

def test_numba_naive(shape, numba_func, dtype=np.float64):
    """ This function compares naive matrix-multiplication with Numba permutation function """

    # 1.
    # Check the shape of matrix
    if shape[0] == shape[1] and shape[1] == shape[2]:
        print('Square matrices, N = %d' %shape[0])
    else:
        print('Non-square matrices, N = %d, K = %d, M = %d' %(shape[0], shape[1], shape[2]))
        
    print()
        
    # 2.
    # Generate random matrices
    A = np.random.rand(shape[0], shape[1])
    B = np.random.rand(shape[1], shape[2])
    
    # 3.
    # Time functions: Naive matrix-multiplication and Numba function
    t1_start = time()
    C1 = my_naive_matmul(A, B)
    t1_end = time()
    t1 = t1_end - t1_start

    t2_start = time()
    C2 = numba_func(A, B)
    t2_end = time()
    t2 = t2_end - t2_start

    # 4.
    # Calculate speed-up and accuracy
    if (t1 >= t2):
        print('%s is faster' %(numba_func.__name__))
        speedup = t1 / t2
    else:
        print('Naive matmul is faster')
        speedup = t2 / t1

    print('Speed-up factor: %.3f' %speedup)
    print('Error: %s' %compare_matmul(C1, C2))
    

def test_numba_float_type(shape, numba_func):
    """ This function compares Numba permutation function for different data types """

    # 1.
    # Check the shape of matrix
    if shape[0] == shape[1] and shape[1] == shape[2]:
        print('Square matrices, N = %d' %shape[0])
    else:
        print('Non-square matrices, N = %d, K = %d, M = %d' %(shape[0], shape[1], shape[2]))
        
    print()
        
    # 2.
    # Generate random matrices
    A = np.random.rand(shape[0], shape[1])
    B = np.random.rand(shape[1], shape[2])
    
    # 3.
    # Time functions: Numba function with different data types
    t1_start = time()
    C1 = numba_func(A.astype(np.float64), B.astype(np.float64))
    t1_end = time()
    t1 = t1_end - t1_start

    t2_start = time()
    C2 = numba_func(A.astype(np.float32), B.astype(np.float32))
    t2_end = time()
    t2 = t2_end - t2_start

    # 4.
    # Calculate speed-up and accuracy
    if (t1 >= t2):
        print('Float32 is faster')
        speedup = t1 / t2
    else:
        print('Float64 is faster')
        speedup = t2 / t1

    print('Speed-up factor: %.3f' %speedup)
    print('Difference in results: %s' %compare_matmul(C1, C2))

<br>

### Write the test codes for Exercise 2 in the cell below.

In [25]:
# Compare with np.dot()

for N in np.array([128, 256, 512]):
    test_numba_matmul([N, N, N], dtype=np.float64)
    print('\n%s\n' %("-" * 75))


Square matrices, N = 128

Error: 0.0

Fastest method: np.dot()
Best Numba permutation: numba_matmul_2
Speed-up over Numba functions: 16.81, 2.80, 13.90, 23.98, 9.56, 24.03

---------------------------------------------------------------------------

Square matrices, N = 256

Error: 0.0

Fastest method: np.dot()
Best Numba permutation: numba_matmul_2
Speed-up over Numba functions: 46.65, 6.10, 41.91, 118.17, 24.92, 120.02

---------------------------------------------------------------------------

Square matrices, N = 512

Error: 1.6109439116007707e-08

Fastest method: np.dot()
Best Numba permutation: numba_matmul_2
Speed-up over Numba functions: 102.49, 12.33, 104.42, 295.45, 42.95, 289.81

---------------------------------------------------------------------------



In [27]:
# Fastest Numba permutation function compared with naive matrix-multiplication

for N in np.array([128, 256, 512]):
    test_numba_naive([N, N, N], numba_matmul_2, dtype=np.float64)
    print('\n%s\n' %("-" * 75))


Square matrices, N = 128

numba_matmul_2 is faster
Speed-up factor: 2574.797
Error: 2.7686297698892304e-11

---------------------------------------------------------------------------

Square matrices, N = 256

numba_matmul_2 is faster
Speed-up factor: 2726.335
Error: 2.1925217197349411e-10

---------------------------------------------------------------------------

Square matrices, N = 512

numba_matmul_2 is faster
Speed-up factor: 2231.254
Error: 1.7816859099184512e-09

---------------------------------------------------------------------------



In [126]:
# Fastest Numba permutation function, compare different float types

for N in np.array([128, 256, 512]):
    test_numba_float_type([N, N, N], numba_matmul_2)
    print('\n%s\n' %("-" * 75))


Square matrices, N = 128

Float32 is faster
Speed-up factor: 1.053
Difference in results: 0.06564263418561822

---------------------------------------------------------------------------

Square matrices, N = 256

Float32 is faster
Speed-up factor: 1.036
Difference in results: 0.7351486192010128

---------------------------------------------------------------------------

Square matrices, N = 512

Float32 is faster
Speed-up factor: 1.008
Difference in results: 8.168771339268773

---------------------------------------------------------------------------



<br>

## Discussion for Exercise 2
1. What do you think is causing the differences in performance between the various permutations?
1. Which function is fastest (permutations or numpy.dot)? Why do you think this function is the fastest, and could there be multiple factors involved regarding the superior performance?
1. When comparing the matrix-matrix performance between the cases were all matrices had dtype=np.float64 vs.  all matrices having dtype=np.float32, which dtype was fastest? Roughly what was the speedup when using this dtype vs the other?
1. Does Amdahl's Law play a major factor in the performance differences? Which part of the matrix-matrix multiplication was not parallelized (serial portion)? (Hint: which matrices did we reuse for each function?) Could we parallelize this part, and if so, are there caveats?
1. Do you think all codes should be parallelized? What about matrix-matrix multiplication? (This is a subjective question, but I am looking for a brief, but rational and informed arguement).
1. For those taking the class for **4 units**: Regarding the performance differences, which Law do you think is more relevant when comparing the performance differences between each of the functions in this exercise, Amdahl's Law or Gustafson's Law? Explain your reasoning. 

## Answer for Exercise 2

1. The CPU cache management should cause the differences in performance. For example, the second case, which is the fastest one, has three loops organized in this way:
```python
for i in numba.prange(N):
    for k in range(K):
        for j in range(M):
            C[i, j] += A[i, k] * B[k, j]
```
Under this case, each matrix is accessed based on the order where the second index is in the inner loop compared to the first index. This is consistent with the 1-D array representation of the matrix, and the cache memory is best used since the new element is always near the previous one. By contrast, for the slowest permutations like $(k, j, i)$ and $(j, k, i)$, the order of indices is contradictory to the one stored in cache, so the cache keeps refreshing the storage.

2. np.dot() is the fastest, because it uses BLAS library (Basic Linear Algebra Subprograms), which contains highly optimized algorithms for vector and matrix operations.

3. Float32 type yields slightly faster computation than float64 type. The speedup factor is just around $1.01$-$1.05$, and appears to be decreasing as the matrix size $N$ goes up.

4. The speedup factor for the fastest permutation function using Numba versus my_naive_matmul() is over $2000$. This indicates that the parallel portion of the program is significant. Based on Amdahl's law, assuming $1/(1-p) = 2000$, the parallel portion of the program is $p = 99.95\%$. This is reasonable, as the original my_naive_matmul() has time complexity $O(N^3)$, and we have quite large matrix size $N$. On the other hand, as the matrix size $N$ goes up, the speedup factor is basically the same. From Amdahl's law, this suggests that the parallel portion $p$ is less influenced by matrix size $N$, given the processor number unchanged. Gustafson's law yields the same conclusion that $p$ does not grow as the problem scales, within the exercise range. The serial portion of the function should include: matrix dimension check, creating zero matrix $C$, and adding results to corresponding entries in $C$ after the element multiplication is done.

5. Not all codes need to be parallelized. First, only the codes that contain parallel portion can be parallelized. A completely serial procedure can not be improved by having more processors, constrained by the order of operations. Second, the amount of parallel portion $p$ matters. Amdahl's law indicates that the maximum speedup factor is $1/(1-p)$. If the serial portion is too large, or the code already optimizes the major parallel portion, then probably you don't need to further make it parallelized. Third, coding time needs to be considered. Widely used subroutines should be parallelized with higher priority than less-called subroutines. For example, matrix multiplication is a basic step in linear algebra that is widely applied in many scientific areas. Therefore, we should consider to parallelize and optimize matrix multiplication, because we use it many times everyday.

6. (Extra) Both laws can be applied to analyze the speedup, since the processor number is never changed (i.e. always on my laptop). Generally, if the problem size is fixed and we study the speedup as a function of processor number, Amdahl's law is relevant. If the problem size (matrix size here) also scales with increasing number of processors, then Gustafson's law is more relevant.
